<a href="https://colab.research.google.com/github/trajinthan/pump-it-up-data-mining/blob/main/pumb_it_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

Authenticate with google drive

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Load data from google drive to colab work space according to the csv file id

In [7]:
import pandas as pd

training_labels = drive.CreateFile({'id':'12QS3xedC7EoPS4Xj2cNVuwBSLnvbJMNM'}) 
training_labels.GetContentFile('TrainLabel.csv')  
Y_train = pd.read_csv('TrainLabel.csv')

training_values = drive.CreateFile({'id':'1F4TZBjMRpTPkEbW7vjpQBIhl7Kp3QlEf'}) 
training_values.GetContentFile('TrainValue.csv')  
X_train = pd.read_csv('TrainValue.csv')

testing_labels = drive.CreateFile({'id':'1Y4Idhc-WeUTM5uQSjZOqyQ5r4ePgUD84'}) 
testing_labels.GetContentFile('TestData.csv')  
Xtest = pd.read_csv('TestData.csv')

In [ ]:
X_train.head().T

In [ ]:
X_train.dtypes